# Lambda Functions for AWS (Amazon Web Services)

This Lambda Functions are the ones responsible for retrieving info from the web and sending it to SQL through Amazon RDS (Relational Database Service)

## flights

In [ ]:
# this code will retrieve flight info through an API, store it on a Data Frame and then send it to SQL

# we start by importing nececessary libraries
import json 
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

# this is the function that will be called by AWS Lambda
def lambda_handler(event, context):
    
    # icao codes of airports we want to retrieve data from
    icao_list = ['LEST', 'EDDB']  
    
    # name of the SQL schema we want to use 
    schema="project_gans"  
    # name of the SQL host
    host="gans-project-db.cnxahfg4n8cc.eu-west-3.rds.amazonaws.com" 
    # name of the SQL user
    user="admin" 
    # password of the SQL user
    password="your_password"
    # port of the SQL host
    port=3306  
    # connection string to connect to SQL
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}' 

    # to get today's date
    today = datetime.now().astimezone(timezone('Europe/Berlin')).date() 
    #  to get tomorrow's date
    tomorrow = (today + timedelta(days=1)) 
    
    # initialize empty lists to store data
    flight_id = []  
    dep_city = []
    dep_icao = []
    dep_time_local = []
    arr_icao = []
    arr_time = []
    arr_terminal = []
    flight_num = []
    airline = []
    
    # initialize an id variable to keep track of the flight_id
    id = 0  
    
    # loop through the icao codes of the airports we want to retrieve data from
    for icao in icao_list: 
        times = [["00:00","11:59"],["12:00","23:59"]]
    
        # loop through the times of the day we want to retrieve data from
        for time in times: 
            # url of the API we want to retrieve data from
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/ \
                {tomorrow}T{time[0]}/{tomorrow}T{time[1]}"  
            # querystring with the parameters we want
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false", 
                           "withCodeshared":"true","withCargo":"false","withPrivate":"false"}
            # headers of the API we want to retrieve data from
            headers = {
              'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
              'x-rapidapi-key': "your_key"
              } 
            
            # send the request to the API and save the response
            response = requests.request("GET", url, headers=headers, params=querystring) 
            # save the response in json format
            r = response.json() 
    
            # loop through the response of the API to extract the data we want
            for i in range(len(r['arrivals'])): 
                # to increment the id variable
                id += 1  
                # to append the id to the flight_id list
                flight_id.append(id) 
                # to append the departure city to the dep_city list
                dep_city.append(r['arrivals'][i]['departure']['airport']['name']) 
                # try this, if it works append the departure airport ICAO code to the dep_icao list
                try: 
                  dep_icao.append(r['arrivals'][i]['departure']['airport']['icao'])
                # if it doesn't work append None to the dep_icao list
                except: 
                  dep_icao.append(None)
                # same here but for departure times
                try: 
                  dep_time_local.append(r['arrivals'][i]['departure']['scheduledTime']['local'])
                except:
                  dep_time_local.append(None)
                # to append icao code of the arrival airport to the arr_icao list
                arr_icao.append(icao) 
                # to append arrival time to the arr_time list
                arr_time.append(r['arrivals'][i]['arrival']['scheduledTime']['local']) 
                # to append arrival terminal to list, if it works
                try: 
                  arr_terminal.append(r['arrivals'][i]['arrival']['terminal'])
                # if it doesn't work append None
                except: 
                  arr_terminal.append(None)
                # to append flight number to the flight_num list
                flight_num.append(r['arrivals'][i]['number'])
                # to append airline name to the airline list
                airline.append(r['arrivals'][i]['airline']['name']) 
    
   
    # create a list of tuples with the data we retrieved from the API
    ap_ct = list(zip(flight_id, dep_city, dep_icao, dep_time_local, arr_icao, arr_time, arr_terminal, flight_num, airline))
    # create a dataframe from that list of tuples, setting the columns names with 'columns' parameter
    airports_cities = pd.DataFrame(ap_ct, columns = ['flight_id', 'dep_city', 'dep_icao', 'dep_time', 'arr_icao', 'arr_time', 
                                                     'arr_terminal', 'flight_num', 'airline'])

    # use .to_sql() to send the dataframe to SQL, if_exists = 'append' to append the data to the table
    # con = con string to connect to SQL, index = False to not include the index in the table
    airports_cities.to_sql('flights', if_exists='append', con=con, index=False) 

    # if everything works correctly, the lambda function will return this, otherwise it will return an error
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


## cities

In [ ]:
# this code retrieves city information from an API and then sends it to SQL

# we start by importing nececessary libraries
import json
import pandas as pd
import requests
from datetime import datetime
import pytz

def lambda_handler(event, context):  # this is the function that will be called by AWS Lambda
    
    # same as before, this is the info needed to connect to SQL
    schema="project_gans"  
    host="gans-project-db.cnxahfg4n8cc.eu-west-3.rds.amazonaws.com"        
    user="admin"
    password="your_password" 
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    # cities we want to retrieve data from
    cities = ['Santiago de Compostela', 'Berlin']
    
    #  API key of the API we want to retrieve data from
    API_key = "your_key" 

    # to set desired time zone
    tz = pytz.timezone('Europe/Berlin') 
    # to get the current time in desired time zone
    now = datetime.now().astimezone(tz) 
    
    # initialiase empty lists to store data
    weather = []
    city_id = []
    city_name = []
    country_code = []
    # initialise an n variable to keep track of the city_id
    n = 0

    # we loop through the cities we want to retrieve data from
    for i in range(len(cities)): 
        # url of the API we want to retrieve data from
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={cities[i]}&appid={API_key}&units=metric")
        # send the request to the API and append the response to the weather list
        weather.append(requests.get(url).json())
        
        # use n to keep track of the city_id, appending it to the city_id list
        n += 1
        city_id.append(n)
        # append the city name to the city_name list
        city_name.append(weather[i]['city']['name'])
        # append the country code to the country_code list
        country_code.append(weather[i]['city']['country'])

    # create a list of tuples with the data we retrieved from the API
    ct = list(zip(city_id, city_name, country_code))
    #  create a dataframe from that list of tuples, setting the name of the columns
    cities_df = pd.DataFrame(ct, columns = ['city_id', 'city_name', 'country_code'])
    
    # use .to_sql() to send the dataframe to SQL, if_exists = 'append' to append the data to the table
    # con = con string to connect to SQL, index = False to not include the index in the table
    cities_df.to_sql('cities', if_exists='append', con=con, index=False)        
    
    # if the function works correctly, the lambda function will return this, otherwise it will return an error
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

## populations

In [ ]:
# this code will retrieve population information from an API and then send it to SQL

# import necessary libraries
import json
import pandas as pd
import requests
from datetime import datetime
import pytz
import sqlalchemy

# this is the function that will be called by AWS Lambda
def lambda_handler(event, context):
    
    # this is the necessary info to connect with SQL
    schema="project_gans"
    host="gans-project-db.cnxahfg4n8cc.eu-west-3.rds.amazonaws.com"
    user="admin"
    password="your_password"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    # the cities we want to retrieve data from
    cities = ['Santiago de Compostela', 'Berlin']
    # API key of the API we want to retrieve data from
    API_key = "your_key"
    
    # set the time zone needed and then get the current time such time zone
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    
    # initialiase an empty list to store the API reponse
    city = []
    # id to keep track of the city_id
    id = 0
    
    # use a for loop to do an API call for each city in the cities list
    for i in cities:
        # make the request to the API
        cities = requests.get(f"https://api.api-ninjas.com/v1/city?name={i}", headers={'X-Api-Key': API_key})
        # save the response in json format
        cities_json = cities.json()
        # append the response to the city list if the request worked correctly
        if cities.status_code == requests.codes.ok:
            city.append(cities_json)
        # otherwise print an error
        else:
            print("Error:", cities.status_code, cities.text)
            break  # exit the loop if an error occurs
    
        # initialiase empty lists to store data
        city_id = []
        population = []
        year = [] 
    
        # this loop will append the data we want to the lists
        for x in range(len(city)):
            city_id.append(id)
            population.append(city[x][0]['population'])
            year.append(now.year)
            id += 1
    
    # create a list of tuples with lists created above
    pp = list(zip(city_id, population, year))
    # create and save a dataframe from that list of tuples, setting the column names with 'columns' parameter
    populations_df = pd.DataFrame(pp, columns = ['city_id', 'population', 'year'])
    
    # use .to_numeric to convert the 'population' column to numeric, and downcast to the smallest float type
    populations_df['population'] = pd.to_numeric(populations_df['population'], downcast = 'float')
    
    # use .to_sql() to send the dataframe to SQL, if_exists = 'append' to append the data to the table
    # con = con string to connect to SQL, index = False to not include the index in the table
    populations_df.to_sql('populations', if_exists='append', con=con, index=False)
    
    #  if the function works correctly, the lambda function will return this, otherwise it will return an error
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


## cities-airports

In [ ]:
# this info will retrieve info from cities and their airports ICAO codes and send it to SQL

# import necessary libraries
import json
import pandas as pd
import requests
from pytz import timezone

# function name that will be called by AWS Lambda
def lambda_handler(event, context):
    
    # info needed to connect with SQL
    schema="project_gans"
    host="gans-project-db.cnxahfg4n8cc.eu-west-3.rds.amazonaws.com"
    user="admin"
    password="your_password"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    # latitudes and longitudes of the cities we want to retrieve data from (Santiago de Compostela and Berlin)
    # we need this because the APIs asks for the latitude and longitude to get the nearest airports
    latitudes = [42.8963, 52.31]
    longitudes = [-8.41514, 13.24]
    
    # initialise empty lists to save data
    response = []
    city_id = []
    airport_icao = []
    # use n variable to keep track of the city_id
    n = 1
    
    # for loop to get tne nearest airports within a 50km radius from the latitudes and longitudes
    for i in range(len(latitudes)):
        #  url of the API we want to retrieve data from
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        # querystring with the paramenters we want to send to the API
        querystring = {"lat":latitudes[i],"lon":longitudes[i],"radiuskm":"50","limit":"10","withFlightInfoOnly":"true"}
        #  headers of the API we want to send the request to
        headers = {
          "X-RapidAPI-Key": "your_key",
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        # append the response to the API request to the response list
        response.append(requests.get(url, headers=headers, params=querystring).json())
    
    # we need two for loops to go through the response list and append the data to the city_id and airport_icao lists
    for outer_idx, item in enumerate(response):
        for inner_idx, airport in enumerate(item['items']):
            city_id.append(n)
            airport_icao.append(airport['icao'])
        n += 1
    
    # create a list of tuples with the city_id and airport_icao lists
    ct_ar = list(zip(city_id, airport_icao))
    # create a data frame from that list of tuples, setting the column names with 'columns' parameter
    cities_airports_df = pd.DataFrame(ct_ar, columns = ['city_id', 'airport_icao'])
    
    # use .to_sql() to send the dataframe to SQL, if_exists = 'append' to append the data to the table
    # con = con string to connect to SQL, index = False to not include the index in the table
    cities_airports_df.to_sql('cities_airports', if_exists='append', con=con, index=False) 

    # if th function works correctly, the lambda function will return this, otherwise it will return an error
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

## weathers-cities

In [ ]:
# this function will get weather information from the openweathermap API and send it to SQL

# import necessary libraries
import json
import pandas as pd
import requests
from datetime import datetime
import pytz

# AWS Lambda function name, this  will be called by AWS Lambda
def lambda_handler(event, context):
    
    # necessary info to connec to SQL
    schema="project_gans"  
    host="gans-project-db.cnxahfg4n8cc.eu-west-3.rds.amazonaws.com"        
    user="admin"
    password="your_password" 
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    # cities we're retrieving info from
    cities = ['Santiago de Compostela', 'Berlin']
    
    # openweathermap API Key
    API_key = "your_key"

    # set desired timezone and current time
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    
    # initialise empty lists to save info, and n variable to keep track of the city_id
    weather = []
    city_id = []
    n = 0

    # this outside for loop will loop through the cities and make an API request to each one
    for i in range(len(cities)):
        # url of the API we want to retrieve data from
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={cities[i]}&appid={API_key}&units=metric")
        # append the response to the API request to the weather list in json format
        weather.append(requests.get(url).json())
        
        # keep track of the city_id and appeend it to the city_id list
        n += 1
        city_id.append(n)
        
        # initialise empty lists to save data
        city_idd = []
        country = []
        city = []
        date = []
        temp_min = []
        temp_max = []
        id = []
        weather_type = []
        weather_desc = []
        rain_P = []
        rain = []
        snow = []
        clouds = []
        humidity = []
        wind = []
    
        # inside for loops to extract data from the response saved in weather
        for i in range(len(weather)): 
            # loop through the list of dictionaries in the response and append the needed data to each list
            for j in range(len(weather[i]['list'])): 
                city_idd.append(city_id[i])
                country.append(weather[i]['city']['country'])
                city.append(weather[i]['city']['name'])
                date.append(weather[i]['list'][j]['dt_txt'])
                temp_min.append(weather[i]['list'][j]['main']['temp_min'])
                temp_max.append(weather[i]['list'][j]['main']['temp_max'])
                id.append(weather[i]['list'][j]['weather'][0]['id'])
                weather_type.append(weather[i]['list'][j]['weather'][0]['main'])
                weather_desc.append(weather[i]['list'][j]['weather'][0]['description'])
                rain_P.append(weather[i]['list'][j]['pop'])
                try:
                    rain.append(weather[i]['list'][j]['rain']['3h'])
                except: # append 0 if there is no rain data
                    rain.append(0)
                try:
                    snow.append(weather[i]['list'][j]['snow']['3h'])
                except: # append 0 if there is no snow data
                    snow.append(0)
                clouds.append(weather[i]['list'][j]['clouds']['all'])
                humidity.append(weather[i]['list'][j]['main']['humidity'])
                wind.append(weather[i]['list'][j]['wind']['speed'])
    
    # create a list of tuples
    wt = list(zip(city_idd, country, city, date, temp_min, temp_max, id, weather_type, weather_desc, 
                  rain_P, rain, snow, clouds, humidity, wind))
    # create a data frame from the list of tuples, setting the column names with 'columns' parameter
    weather_df = pd.DataFrame(wt, columns = ['city_id', 'country', 'city', 'date_time', 'min_temp', 'max_temp', 
                                           'w_id', 'w_type', 'w_desc', 'rain_P', 'rain_mm', 'snow_mm', 'clouds', 
                                           'humidity', 'wind'])
              
    # use .to_sql  to send the dataframe to SQL, if_exists = 'append' to append the data to the table
    # con = con string to connect to SQL, index = False to not include the index in the table
    weather_df.to_sql('weathers', if_exists = 'append', con = con, index = False)  
    
    # if the function works correctly, the lambda function will return this, otherwise it will return an error
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

## airports

In [ ]:
# this function gets a .cvs file that contains the airports of the world and then sends it to SQL

# first, import necessary libraries
import pandas as pd
import requests
from io import StringIO # string IO is a library that allows us to read the .csv file as a string

# AWS Lambda function name, this  will be called by AWS Lambda
def lambda_handler(event, context):
  
    # url that contains the .csv file
    url = 'https://raw.githubusercontent.com/ip2location/ip2location-iata-icao/master/iata-icao.csv'
    
    # necessary info to connect with SQL
    schema="project_gans"
    host="gans-project-db.cnxahfg4n8cc.eu-west-3.rds.amazonaws.com"
    user="admin"
    password="your_password"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # make request to retrieve the .csv file
    table_url = url
    response = requests.get(table_url)
    
    # if the response works correctly, the status code will be 200, otherwise it will return an error
    if response.status_code == 200:
        table_data = response.text
    else:
        print("Failed to retrieve the table data. Status code:", response.status_code)
    
    # convert the .csv file to a string
    csv_io = StringIO(table_data) 
    # read the .csv file as a data frame
    df = pd.read_csv(csv_io)
    # create a data frame
    airports_df = pd.DataFrame(df)
    # drop duplicates
    airports_df = airports_df.drop_duplicates()
    # drop duplicates based on the 'icao' column
    airports_df = airports_df.drop_duplicates('icao') 
    # drop rows with missing values
    airports_df = airports_df.dropna() 
    # rename columns
    airports_df = airports_df.rename(columns = {'iata' : 'airport_iata', 'icao' : 'airport_icao', 
                                                'airport' : 'airport_name'})
    # reorder columns
    airports_df = airports_df[['airport_icao', 'airport_name', 'country_code', 'region_name', 'airport_iata', 
                               'latitude', 'longitude']] 
    
    # use .to_sql  to send the dataframe to SQL, if_exists = 'append' to append the data to the table
    # con = con string to connect to SQL, index = False to not include the index in the table
    airports_df.to_sql('airports', if_exists = 'append', con = con, index = False) 
              
    return {'statusCode': 200}